# Introduction

The aim of this project is to use SQL to obtain data from a relational database with 11 different tables containing information about a record store, and answer some questions about the data.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


**Which genres sell the best in the USA**

The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first 3 albums added to the store, out of the four.  The record label specialises in artists from the USA, so we're interested in finding out which genres sell the best in the USA.

The artists are as follows:


Artist Name	            |   Genre
------------------------|---------
Regal	                |  Hip-Hop
Red Tone	            |   Punk
Meteor and the Girls	|    Pop
Slim Jim Bites	        |   Blues

In [46]:
%%sql
WITH country_usa AS (SELECT
                    customer_id,
                    country
                    FROM customer
                    WHERE country = 'USA'),

     tracks_sold_us AS (SELECT
                       c.customer_id,
                       i.invoice_id,
                       il.track_id,
                       il.quantity AS total_purchases
                       FROM country_usa c
                       INNER JOIN invoice i ON i.customer_id = c.customer_id
                       INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id),
    tracks_by_genre AS (SELECT
                        ts.track_id,
                        t.genre_id,
                        g.name,
                        SUM(ts.total_purchases) total_purchases
                        FROM tracks_sold_us ts
                        INNER JOIN track t ON t.track_id = ts.track_id
                        INNER JOIN genre g ON g.genre_id = t.genre_id
                        GROUP BY ts.total_purchases, g.name, g.genre_id)
    
SELECT 
name,
total_purchases,
ROUND(CAST(total_purchases AS FLOAT) * 100 / 
      (SELECT SUM(total_purchases) FROM tracks_by_genre),2) AS percentage_purchased
FROM tracks_by_genre
GROUP BY total_purchases
ORDER BY total_purchases DESC
LIMIT 100;
                       

Done.


name,total_purchases,percentage_purchased
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33
Easy Listening,13,1.24


It seems as if Rock was far and away the most popular genre in the US, however given the lists of artists we have I believe Red Tone would be the first choice as their genre is Punk, followed by Slim Jim Bites who are Blues and lastly, Meteor and the Girls being a Pop band.

**Total Sales for Each Sales Support Agent**

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase.  Our task is to analyse the purchases belonging to each employee to see which sales agent is performing better or worse than the others, and try to identify a reason for their performance.

In [65]:
%%sql
WITH employee_info AS (SELECT
                                employee_id,
                                first_name || ' ' || last_name AS employee_name,
                                hire_date
                                FROM employee
                                WHERE title = 'Sales Support Agent'),


    customer_info AS (SELECT
                      ei.*,
                      c.customer_id
                      FROM employee_info ei
                      INNER JOIN customer c ON c.support_rep_id = ei.employee_id
                      )
    
SELECT
ci.employee_name,
ci.hire_date,
COUNT(ci.customer_id) AS number_of_customers,
SUM(i.total) AS total_purchases
FROM customer_info ci
INNER JOIN invoice i ON i.customer_id = ci.customer_id
GROUP BY ci.employee_name


Done.


employee_name,hire_date,number_of_customers,total_purchases
Jane Peacock,2017-04-01 00:00:00,212,1731.5100000000039
Margaret Park,2017-05-03 00:00:00,214,1584.0000000000034
Steve Johnson,2017-10-17 00:00:00,188,1393.920000000002


We can see that Jane made the most money, followed by Margaret, and then steve which roughly corresponds with their hire dates.  Additionally, there's a slight correlation between the number of customers and total sales, as Steve has the least customers assigned to him, and the least sales.

**Analysing Sales Data from customers from each different country**

Next, we'll analyse the sales data for each country, on the:

* Total number of customers

* Total value of sales

* Average value of sales per customer

* Average order value

And determine if there are any markets that are useful to invest into.

In [115]:
%%sql

WITH grouped_country AS (SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 WHERE country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
     )

SELECT
country,
total_customers,
total_sales,
average_customer_value,
average_order_value
FROM
(SELECT 
country,
COUNT(DISTINCT(customer_id)) AS total_customers,
SUM(unit_price) AS total_sales,
SUM(unit_price) / COUNT(DISTINCT(customer_id)) AS average_customer_value,
SUM(unit_price) / COUNT(DISTINCT(invoice_id)) AS average_order_value,
CASE
WHEN country = 'Other' THEN 1
ELSE 0
END AS sort
FROM grouped_country
GROUP BY country
ORDER BY total_sales DESC)
ORDER BY sort ASC;
                         

Done.


country,total_customers,total_sales,average_customer_value,average_order_value
USA,13,1040.490000000008,80.03769230769292,7.942671755725252
Canada,8,535.5900000000034,66.94875000000043,7.047236842105309
Brazil,5,427.68000000000245,85.53600000000048,7.011147540983647
France,5,389.0700000000021,77.81400000000042,7.781400000000042
Germany,4,334.6200000000016,83.6550000000004,8.161463414634186
Czech Republic,2,273.24000000000103,136.62000000000052,9.108000000000034
United Kingdom,3,245.52000000000078,81.84000000000026,8.768571428571457
Portugal,2,185.13000000000022,92.56500000000011,6.3837931034482835
India,2,183.1500000000002,91.5750000000001,8.72142857142858
Other,15,1094.9400000000085,72.99600000000056,7.448571428571486


Looking at the average order values the UK, India and Czech Republic seem like good markets to break into, as customer value is good and the average order value is high, so likely the customers will be more valuable.